Analying Customer churn for a marketing agency. Using a Logistic Regression Model in Spark to create a classification algorithm that predicts customer churn.

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('CustomerChurn').getOrCreate()

In [5]:
data = spark.read.csv('../../SparkData/customer_churn.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [9]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
from pyspark.ml.feature import VectorAssembler

In [13]:
# choose the features to transform in the assembler
assembler = VectorAssembler(inputCols=[
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],
    outputCol='features')

In [14]:
output = assembler.transform(data)

In [15]:
final_data = output.select('features', 'churn')

In [16]:
final_data.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [17]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [18]:
from pyspark.ml.classification import LogisticRegression

In [19]:
lr_churn = LogisticRegression(labelCol='churn')

In [20]:
# fit the training data
fitted_churn_model = lr_churn.fit(train_churn)

In [21]:
training_summary = fitted_churn_model.summary

In [22]:
training_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                631|                631|
|   mean|0.15530903328050713|0.11410459587955626|
| stddev| 0.3624863757450892| 0.3181904917535037|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
# evaluate the model on the testing data
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [25]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,9090.43,1.0...|    0|[1.85198432930934...|[0.86435991768188...|       0.0|
|[28.0,11128.95,1....|    0|[4.37872530993949...|[0.98761400219336...|       0.0|
|[28.0,11204.23,0....|    0|[1.84273641812697...|[0.86327201746214...|       0.0|
|[28.0,11245.38,0....|    0|[3.42777981357594...|[0.96856153315973...|       0.0|
|[29.0,10203.18,1....|    0|[3.96641315817646...|[0.98141085102397...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.70810768787819...|[0.97606313862680...|       0.0|
|[30.0,8403.78,1.0...|    0|[6.27597785495765...|[0.99812258047677...|       0.0|
|[30.0,10960.52,1....|    0|[2.59249169350761...|[0.93037679179006...|       0.0|
|[31.0,8688.21,0.0...|    0|[6.58581353682312...|[0.99862209684762...|       0.0|
|[31.0,11297.57,

In [26]:
# specify the prediction and label columns
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [27]:
# evaluate the ROC/AUC of the predictions column
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [28]:
auc

0.738479262672811

The AUC is decent. A random guess produces an AUC of 0.5, thus our model has performed much better than the baseline.

***

In [29]:
final_lr_model = lr_churn.fit(final_data)

In [30]:
new_customers = spark.read.csv('../../SparkData/new_customers.csv', 
                               inferSchema=True, header=True)

In [31]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [32]:
test_new_customers = assembler.transform(new_customers)

In [33]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [35]:
final_results = final_lr_model.transform(test_new_customers)

In [37]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

